In [ ]:
import os

In [ ]:
directory = "results/classifications/20210427/"

### Rename files

In [ ]:
# for file in os.listdir(directory):
#     #print(file[0:58]+'20201215.tif')
#     os.rename(directory+file, directory+file[0:58]+'20201215.tif')

## Mosiac tifs

In [ ]:
#build vrt
os.chdir(directory)
os.system("gdalbuildvrt " + "Eastern_gm_mads_two_seasons_20210427_mosaic.vrt *.tif")

#build mosaic from .vrt
os.system("gdal_translate "\
       "-co BIGTIFF=YES "\
       "-co COMPRESS=DEFLATE "\
       "-co ZLEVEL=9 "\
       "-co PREDICTOR=1 "\
       "-co TILED=YES "\
       "-co BLOCKXSIZE=1024 "\
       "-co BLOCKYSIZE=1024 "\
       + "Eastern_gm_mads_two_seasons_20210427_mosaic.vrt " + "Eastern_gm_mads_two_seasons_20210427_mosaic.tif")

## Clip mosaic to AEZ

In [ ]:
# !pip install -e ../../production/dea_ml
# !pip install git+https://github.com/digitalearthafrica/deafrica-sandbox-notebooks.git@minty-fresh-sandbox#subdirectory=Tools

In [ ]:
import xarray as xr
import numpy as np
import geopandas as gpd
from datacube.utils.cog import write_cog
import sys

from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.dask import create_local_dask_cluster

In [ ]:
create_local_dask_cluster()

In [ ]:
# aez_region = 'data/Eastern.shp'
aez_region = 'data/WDPA_eastern.shp'
pred_tif = "results/classifications/20210427/Eastern_gm_mads_two_seasons_20210427_mosaic_clipped.tif"
chunks={'x':10000, 'y':10000}

In [ ]:
aez = gpd.read_file(aez_region)
da = xr.open_rasterio(pred_tif)

In [ ]:
aez = xr_rasterize(aez, da)

In [ ]:
aez = aez.astype(bool)

In [ ]:
da = da.chunk(chunks).where(~aez.chunk(chunks)).astype(np.int8)

In [ ]:
%%time
da = da.compute()

In [ ]:
da

In [ ]:
write_cog(da.squeeze(),
          pred_tif[:-4]+'_WDPA_masked.tif',
          overwrite=True)

In [ ]:
# write_cog(aez.squeeze(),
#           'data/WDPA_eastern.tif',
#           overwrite=True)